In [1]:
#Import the Necessary Libraries:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tkinter import Tk, filedialog


In [2]:
#Read Data in:
AUDI_Customer_Data = filedialog.askopenfilename(title="Select CSV file", filetypes=[("CSV files", "*.csv")])
#We're doing an analysis on Audi customers
Audi_Data = pd.read_csv(AUDI_Customer_Data) #read in file 'Data.csv' 
#Now we'd need to look at what are the features, and what is the dependent variable vector based on our data set
    #features are our predecitive measures: i.e., country, age, and salary
    #our dependent variable is S-Line purchases - usually is the last column
X = Audi_Data.iloc[:, :-1].values # mean get all values except ones in last column (our predictive measures)
y = Audi_Data.iloc[:,-1].values # gets values for last column! Will they buy, yes or no.


In [7]:
print(X)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 63777.77777777778]
 ['France' 35.0 58000.0]
 ['Spain' 38.77777777777778 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


In [ ]:
print(y)

In [5]:
#How to deal with MISSING DATA NOW: 
    #we can delete missing data if dataset is big enough
    #but if not, replace missing data using average of collective column

#In our case, we're missing a data point for salary:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean') #replaces missing values with column mean
imputer.fit(X[:,1:3]) # this selects all the columns with numerical values...we cant use this method for non-numerical
X[:, 1:3] = imputer.transform(X[:, 1:3])


In [6]:
print(X)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 63777.77777777778]
 ['France' 35.0 58000.0]
 ['Spain' 38.77777777777778 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


In [8]:
#Encoding Categorical Data:
#0, 1, 2 doesn't work for machine learning as it can potentially priortize said category based on numerical value still.
#so instead we create binary vetcors for all the categories, in seperate columns (1 hot)
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0] )], remainder= 'passthrough')

X = np.array(ct.fit_transform(X))

print(X)


[[1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [0.0 1.0 0.0 30.0 54000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 35.0 58000.0]
 [0.0 0.0 1.0 38.77777777777778 52000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 37.0 67000.0]]


In [9]:
#Encoding for the Dependent Variable (in this case its binary so we can do 0 and 1)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

print(y)

[0 1 0 0 1 1 0 1 0 1]


In [10]:
#NOW let's split the dataset into the Training and Test Sets (80/20)
#Q: do we apply feature scaling before or after splitting the dataset??
    #A: APPLY AFTER SPLITING - why: because a true test set would not have feature scaling done to it prior to model development
        #if we do FS before, it will screw up the calculation of the mean and stdev too - information leakage

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1) #splitting data via 80/20% split; the random state just ensures the same random selection each time.

In [11]:
print(X_train)

[[0.0 0.0 1.0 38.77777777777778 52000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 35.0 58000.0]]


In [12]:
print(X_test)

[[0.0 1.0 0.0 30.0 54000.0]
 [1.0 0.0 0.0 37.0 67000.0]]


In [13]:
print(y_train)

[0 1 0 0 1 1 0 1]


In [14]:
print(y_test)

[0 1]


In [15]:
#Feature Scaling: 
#Normalization/Standardization of data so everything is on the same playing field
#(for linear regresssions) it is sometimes not necessary because the x coefficient scales each term.
#standardization is better for all situations, normalization only really works when you have a normal distribution
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
#we do not need to apply feature scaling to our dumby variables that represent countries...if we did we'd not have a clue what country = what
X_train[:, 3:] = sc.fit_transform(X_train[:, 3:])
    #Now transform test set
X_test[:, 3:] = sc.transform(X_test[:, 3:]) #don't need fit for test set. 

In [16]:
print(X_train)

[[0.0 0.0 1.0 -0.19159184384578545 -1.0781259408412425]
 [0.0 1.0 0.0 -0.014117293757057777 -0.07013167641635372]
 [1.0 0.0 0.0 0.566708506533324 0.633562432710455]
 [0.0 0.0 1.0 -0.30453019390224867 -0.30786617274297867]
 [0.0 0.0 1.0 -1.9018011447007988 -1.420463615551582]
 [1.0 0.0 0.0 1.1475343068237058 1.232653363453549]
 [0.0 1.0 0.0 1.4379472069688968 1.5749910381638885]
 [1.0 0.0 0.0 -0.7401495441200351 -0.5646194287757332]]


In [17]:
print(X_test)

[[0.0 1.0 0.0 -1.4661817944830124 -0.9069571034860727]
 [1.0 0.0 0.0 -0.44973664397484414 0.2056403393225306]]
